# Developing Model Baselines + Feature Engineering

## Setup and Load Data

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold, cross_val_score
from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report, f1_score
import lightgbm as lgb
import xgboost as xgb
import catboost as cat
import time, os, sys
import wandb
from datetime import datetime

In [4]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000):
        display(df)

In [5]:
from src.config import PROJECT_PATH, DATA_PATH, USE_WANDB, WANDB_PROJECT, WANDB_ENTITY
from src.tracking import ExperimentTracker

In [6]:
# Initialize the experiment tracker
tracker = ExperimentTracker(
    project_path=PROJECT_PATH,
    use_wandb=USE_WANDB,
    wandb_project_name=WANDB_PROJECT,
    wandb_entity=WANDB_ENTITY
)

In [7]:
# Load training data
df_train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
df_train_demos = pd.read_csv(os.path.join(DATA_PATH, 'train_demographics.csv'))

In [12]:
# --- Create Helper Mappings for Evaluation Metric ---
# Important for the custom F1 score function
metadata = df_train[['gesture', 'sequence_type']].drop_duplicates()

# Map gesture string to sequence type (Target vs. Non-Target)
gesture_to_seq_type_map = metadata.set_index('gesture')['sequence_type'].to_dict()

# Map gesture string to integer code and back
gesture_map = {label: i for i, label in enumerate(metadata['gesture'].unique())}
inv_gesture_map = {i: label for label, i in gesture_map.items()}

# Validate
print(f"Gesture Map: {gesture_map}")
print(f"\nInverted Gesture Map: {inv_gesture_map}")
print(f"\nGesture To Sequence Type: {gesture_to_seq_type_map}")


Gesture Map: {'Cheek - pinch skin': 0, 'Forehead - pull hairline': 1, 'Write name on leg': 2, 'Feel around in tray and pull out an object': 3, 'Neck - scratch': 4, 'Neck - pinch skin': 5, 'Eyelash - pull hair': 6, 'Eyebrow - pull hair': 7, 'Forehead - scratch': 8, 'Above ear - pull hair': 9, 'Wave hello': 10, 'Write name in air': 11, 'Text on phone': 12, 'Pull air toward your face': 13, 'Pinch knee/leg skin': 14, 'Scratch knee/leg skin': 15, 'Drink from bottle/cup': 16, 'Glasses on/off': 17}

Inverted Gesture Map: {0: 'Cheek - pinch skin', 1: 'Forehead - pull hairline', 2: 'Write name on leg', 3: 'Feel around in tray and pull out an object', 4: 'Neck - scratch', 5: 'Neck - pinch skin', 6: 'Eyelash - pull hair', 7: 'Eyebrow - pull hair', 8: 'Forehead - scratch', 9: 'Above ear - pull hair', 10: 'Wave hello', 11: 'Write name in air', 12: 'Text on phone', 13: 'Pull air toward your face', 14: 'Pinch knee/leg skin', 15: 'Scratch knee/leg skin', 16: 'Drink from bottle/cup', 17: 'Glasses on/of

In [ ]:
FEATURE_WAVE = "Wave 1 - Revised" # Will be updated for each wave run
N_SPLITS = 5
SEED = 42

MODEL_PARAMS = {
    'catboost': {
        'objective': 'MultiClass', # CatBoost specific
        'loss_function': 'MultiClass',
        'eval_metric': 'MultiClass',
        'iterations': 1000,
        'learning_rate': 0.05,
        'depth': 6,
        'l2_leaf_reg': 3.0,
        'random_seed': SEED,
        'verbose': False,
        'allow_writing_files': False
    },
        'lightgbm': {
        'objective': 'multiclass',
        'num_class': 18, # Assuming 18 classes, adjust if different
        'metric': 'multi_logloss',
        'n_estimators': 1000,
        'learning_rate': 0.05,
        'num_leaves': 31,
        'lambda_l1': 0.1,
        'lambda_l2': 0.1,
        'bagging_fraction': 0.8,
        'feature_fraction': 0.8,
        'bagging_freq': 1,
        'verbose': -1,
        'random_state': SEED,
        'n_jobs': -1
    }
}

In [ ]:
# Revised Competition Metric for Primary Evaluation
def average_f1_score(y_true_encoded, y_pred_proba):
    """
    Calculates the competition F1 score using global maps.
    Assumes inv_gesture_map and gesture_to_seq_type_map are globally defined.
    """
    # Check for globals
    if 'inv_gesture_map' not in globals() or 'gesture_to_seq_type_map' not in globals():
        raise ValueError("Global maps 'inv_gesture_map' and 'gesture_to_seq_type_map' are required for average_f1_score.")
    
    y_pred_encoded = np.argmax(y_pred_proba, axis=1)
    
    # Convert encoded labels back to strings
    y_true_str = pd.Series(y_true_encoded).map(inv_gesture_map)
    y_pred_str = pd.Series(y_pred_encoded).map(inv_gesture_map)
    
    # Check for and handle potential mapping errors
    if y_true_str.isnull().any() or y_pred_str.isnull().any():
        raise ValueError("Error in label decoding. Check 'inv_gesture_map'.")
    
    # Binary F1-Calculation
    y_true_binary = y_true_str.map(gesture_to_seq_type_map)
    y_pred_binary = y_pred_str.map(gesture_to_seq_type_map)
    # Debugging checks for invalid sequence types
    invalid_types = set(y_true_binary.unique()) | set(y_pred_binary.unique())
    valid_types = {'Target', 'Non-Target'}
    if not invalid_types.issubset(valid_types):
        raise ValueError(f"Invalid sequence types found: {invalid_types - valid_types}")
    
    binary_f1 = f1_score(y_true_binary, y_pred_binary, 
                         pos_label='Target', average='binary',
                         zero_division=0)
    # Macro F1 calculation (collapsing non targets)
    def collapse_non_target(gesture):
        return 'non_target' if gesture_to_seq_type_map.get(gesture) == 'Non-Target' else gesture
    
    y_true_collapsed = y_true_str.apply(collapse_non_target)
    y_pred_collapsed = y_pred_str.apply(collapse_non_target)
    # Debugging check for invalid gestures after collapse
    invalid_collapsed = set(y_true_collapsed.unique()) | set(y_pred_collapsed.unique())
    valid_collapsed = set(gesture_to_seq_type_map.keys()) | {'non_target'}
    if not invalid_collapsed.issubset(valid_collapsed):
        raise ValueError(f"Invalid collapsed gesture types found: {invalid_collapsed - valid_collapsed}")
    
    macro_f1 = f1_score(y_true_collapsed, y_pred_collapsed, average='macro', zero_division=0)
    return (binary_f1 + macro_f1) / 2

### Wave 0

- Optional: Add a feature indicating percentage of invalid ToF readings per sequence. This can be added later as part of a more advanced wave
- Validate scoring function gesture names.
- Validate all training columns.

In [14]:
def create_wave0_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Creates Wave 0 features: Basic statistical aggregates per sequence.
    
    This function calculates mean, std, min, max, and median for sensor data
    grouped by sequence_id. It also retains metadata like subject and gesture.
    
    Args:
        df (pd.DataFrame): The raw training or test dataframe.
        
    Returns:
        pd.DataFrame: A dataframe with engineered features, one row per sequence_id.
                      Includes 'subject', 'gesture' (for train), and 'gesture_encoded'.
    """
    print("Starting Wave 0 Feature Engineering...")
    
    # Identify sensor columns (adjust prefixes as per your data)
    imu_cols = [c for c in df.columns if c.startswith(('acc_', 'rot_'))]
    tof_cols = [c for c in df.columns if c.startswith('tof_')]
    thm_cols = [c for c in df.columns if c.startswith('thm_')]
    sensor_cols = imu_cols + tof_cols + thm_cols
    
    # --- Robust -1 Handling for ToF ---
    # Replace -1 with NaN in ToF columns before aggregation
    df_proc = df.copy()
    df_proc[tof_cols] = df_proc[tof_cols].replace(-1.0, np.nan)
    
    # Optional: Add a feature indicating percentage of invalid ToF readings per sequence
    # This can be added later as part of a more advanced wave
    
    # Define aggregation functions
    agg_funcs = ['mean', 'std', 'min', 'max', 'median']
    
    # Perform groupby and aggregation
    # Grouping only by 'sequence_id' for Wave 0 (not phase-specific yet)
    # Using 'first' for metadata to get the value from the first row of the group
    agg_dict = {col: agg_funcs for col in sensor_cols}
    meta_cols = [SUBJECT_COL, TARGET_COL] # Include TARGET_COL for training data
    for meta_col in meta_cols:
        if meta_col in df_proc.columns:
             agg_dict[meta_col] = 'first' 
            
    features_df = df_proc.groupby(SEQUENCE_ID_COL).agg(agg_dict).reset_index()
    
    # Flatten MultiIndex columns
    new_columns = [SEQUENCE_ID_COL]
    for col_name, func_name in features_df.columns[1:]: # Skip sequence_id
        # Handle cases where func_name might be a tuple (e.g., from 'first')
        if isinstance(func_name, tuple):
            suffix = "_".join(str(x) for x in func_name if x)
        else:
            suffix = str(func_name)
        new_col_name = f"{col_name}_{suffix}"
        new_columns.append(new_col_name)
    features_df.columns = new_columns
    
    # Rename 'gesture_first' back to 'gesture' for clarity
    if f'{TARGET_COL}_first' in features_df.columns:
        features_df.rename(columns={f'{TARGET_COL}_first': TARGET_COL}, inplace=True)
        
    # Rename 'subject_first' back to 'subject'
    if f'{SUBJECT_COL}_first' in features_df.columns:
        features_df.rename(columns={f'{SUBJECT_COL}_first': SUBJECT_COL}, inplace=True)
        
    # --- Ensure Data Types ---
    # Make sure 'subject' is treated as a category/grouping variable if needed later
    # 'gesture' should remain as is (string/object) for the competition metric
    
    print(f"Feature engineering complete. Shape of features: {features_df.shape}")
    logging.info(f"Wave 0 features created. Shape: {features_df.shape}")
    return features_df

In [15]:
# Generate Wave 0 features
features_df_wave0 = create_wave0_features(df_train)   
display(features_df_wave0.head())

Starting Wave 0 Feature Engineering...
Feature engineering complete. Shape of features: (8151, 1663)


,sequence_id,acc_x_mean,acc_x_std,acc_x_min,acc_x_max,acc_x_median,acc_y_mean,acc_y_std,acc_y_min,acc_y_max,...,thm_4_min,thm_4_max,thm_4_median,thm_5_mean,thm_5_std,thm_5_min,thm_5_max,thm_5_median,subject,gesture
0,SEQ_000007,6.153098,1.334155,3.613281,9.015625,6.488281,3.915570,3.048287,-2.019531,6.519531,...,28.592863,29.761480,29.124386,27.957446,0.877846,26.047148,29.428299,28.181814,SUBJ_059520,Cheek - pinch skin
1,SEQ_000008,3.400506,1.087142,1.734375,5.906250,3.437500,5.311179,3.268073,-0.222656,8.667969,...,28.755495,31.613327,30.809861,25.824221,1.165940,24.181562,28.054575,25.127313,SUBJ_020948,Forehead - pull hairline
2,SEQ_000013,-7.058962,1.295184,-9.250000,-3.347656,-7.144531,2.346182,2.564639,-3.273438,4.683594,...,24.419798,26.452927,24.666821,24.733322,0.475044,24.167980,26.051331,24.594648,SUBJ_040282,Cheek - pinch skin
3,SEQ_000016,5.524654,1.074108,3.437500,9.378906,5.390625,-4.408491,0.598318,-5.718750,-2.960938,...,27.227589,35.665222,34.618870,30.860562,3.310154,26.312038,35.801083,30.702021,SUBJ_052342,Write name on leg
4,SEQ_000018,5.363715,1.627637,1.964844,6.832031,6.101562,4.109737,3.525304,-3.164062,6.718750,...,26.827133,28.400864,27.972450,31.014364,1.394629,28.282324,32.180752,31.771284,SUBJ_032165,Forehead - pull hairline


### Model Training Func - By Kaggle